# 16장 - 순환 신경망으로 순차 데이터 모델링 (part 2)

In [1]:
from IPython.display import Image

# 두 번째 프로젝트: 텐서플로로 글자 단위 언어 모델 구현

언어 모델링(language modeling)은 영어 문장 생성처럼 기계가 사람의 언어와 관련된 작업을 수행하도록 만드는 흥미로운 애플리케이션이다.  
 
앞으로 만들 모델의 입력은 텍스트 문서이다. 입력 문서와 비슷한 스타일로 새로운 텍스트를 생성하는 모델을 만드는 것이 목표이다. 입력 데이터는 책이나 특정 프로그래밍 언어로 만든 컴퓨터 프로그램일 수 있다.  

글자 단위 언어 모델링에서 입력은 글자의 시퀀스로 나뉘어 한 번에 글자 하나씩 네트워크에 주입된다.  
이 네트워크는 지금까지 본 글자와 함께 새로운 글자를 처리하여 다음 글자를 예측한다.

In [2]:
Image(url='https://git.io/JLdVE', width=700)

위의 그림은 글자 단위 언어 모델링의 예를 보여준다. (EOS는 시퀀스의 끝(end of sequence)을 의미한다.)  

이 구현을 데이터 전처리, RNN 모델 구성, 다음 글자를 예측하고 새로운 텍스트를 생성하는 세 개의 단계로 나눈다.  

#### 데이터셋 전처리  
글자 수준의 언어 모델링을 위한 데이터를 준비한다.  
수천 권의 무료 전자책을 제공하는 구텐베르크(Gutenberg) 프로젝트 웹 사이트에서 입력 데이터를 구한다.  
이 예제에서는 쥘 베른이 1974년 출간한 <신비한 섬(The Mysterious Island)> 책의 텍스트를 사용한다.

In [6]:
! curl -O http://www.gutenberg.org/files/1268/1268-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


이 데이터를 내려받으면 보통의 텍스트로 파이썬에서 읽을 수 있다.  
다음 코드에서 다운로드 파일을 직접 읽어 시작과 끝부분을 삭제한다.(구텐베르크 프로젝트에 대한 설명 부분)  
그다음 파이썬 변수 `char_set`을 만들어 이 텍스트에 있는 고유한 단어 집합을 저장.  

In [7]:
import numpy as np


## 텍스트 읽고 전처리하기
with open('/Users/hanhyeongu/Desktop/HG/code study/ML_DL_study/ch16/1268-0.txt', 'r', encoding='UTF8') as fp:
    text=fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
print(start_indx, end_indx)

text = text[start_indx:end_indx]
char_set = set(text)
print('전체 길이:', len(text))
print('고유한 문자:', len(char_set))

567 1112917
전체 길이: 1112350
고유한 문자: 80


텍스트를 내려받고 전처리하여 총 111만 2,350개의 문자와 80개의 고유한 문자로 구성된 시퀀스를 얻었다. 하지만 대부분의 신경망 라이브러리와 RNN 구현은 문자열 형태의 입력 데이터를 다룰 수 없다. 이 때문에 텍스트 데이터를 숫자 형태로 바꾸어야 한다. 이를 위해 파이썬 딕셔너리 `char2int`를 만들어 각 문자를 정수로 매핑한다.  
또한, 모델의 출력 결과를 텍스트로 변환하는 역 매핑도 필요하다. 정수와 문자를 키와 값으로 연결한 딕셔너리로 역 매핑을 수행할 수도 있지만 익덱스와 고유 문자를 매핑한 넘파이 배열을 사용하는 것이 훨씬 효율적이다.  
다음 그림은 "Hello"와 "world"를 사용해서 문자를 정수로 변환하고 그 반대로 변환하는 예를 보여준다.

In [8]:
Image(url='https://git.io/JLdVz', width=700)

In [9]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('인코딩된 텍스트 크기: ', text_encoded.shape)

print(text[:15], '     == 인코딩 ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == 디코딩 ==> ', ''.join(char_array[text_encoded[15:21]]))

인코딩된 텍스트 크기:  (1112350,)
THE MYSTERIOUS       == 인코딩 ==>  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]  == 디코딩 ==>  ISLAND


넘파이 배열 `text_encoded`는 텍스트에 있는 모든 문자에 대한 인코딩 값을 담고 있다. 이 배열을 사용하여 텐서플로 데이터셋을 만들자.

In [11]:
import tensorflow as tf


ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


지금까지 텍스트에 나타난 순서대로 문자를 담기 위해 반복 가능한 Dataset 객체를 만들었다.  
이제 한걸음 물러서서 앞으로 하려는 일에 대해 큰 그림을 그려보자. 텍스트 생성 작업의 경우 이를 분류 작업으로 표현할 수 있다.   

다음 그림에서 불완전한 문자 시퀀스 집합이 있다고 가정해보자.

In [12]:
Image(url='https://git.io/JLdVV', width=700)

위의 그림에서 왼쪽 박스에 있는 시퀀스를 입력으로 생각할 수 있다. 새로운 텍스트를 생성하기 위해 입력 시퀀스가 주어졌을 때 다음 문자를 예측하는 모델을 만드는 것이 목표이다.  
이 입력 시퀀스는 불완전한 텍스트이다. 예를 들어 "**Deep Learn**"을 주입한 후 모델은 다음 문자로 "**i**"를 예측해야 한다.  
80개의 고유한 문자가 있으므로 이 문제는 다중 분류 작업이 된다. 

In [13]:
Image(url='https://git.io/JLdVr', width=700)

위의 그림에서 다중 분류 방식을 기반으로 길이가 1인 시퀀스(즉, 하나의 글자)로 시작해서 새로운 텍스트를 반복하여 생성할 수 있다.  

텐서플로로 텍스트 생성 모델을 구현하기 위해 먼저 시퀀스 길이를 40으로 자른다. 즉, 입력 텐서 x가 40개의 토큰으로 구성된다는 의미이다.  
실제로 시퀀스 길이는 생성된 텍스트의 품질에 영향을 미친다. 긴 시퀀스가 더 의미 있는 문장을 만들 수 있다. 하지만 짧은 시퀀스일 경우 모델이 대부분 문맥을 무시하고 개별 단어를 정확히 감지하는 데 초점을 맞출 수 있다.  
긴 시퀀스가 보통 더 의미 있는 문장을 만들지만 이전에 언급한 것처럼 긴 시퀀스에서 RNN 모델이 장기간 의존성을 감지하기 어렵다. 따라서 실제로 적절한 시퀀스 길이를 찾는 것은 경험적으로 평가해야 하는 하이퍼파라미터 최적화 문제이다. 여기서는 적절한 균형을 유지하기 위해 40을 선택했다.  

위의 그림에서 볼 수 있듯이 입력x와 타깃y는 한 글자씩 어긋나 있다. 따라서 텍스트를 41 문자씩 나눈다. 처음부터 40개의 문자는 입력 시퀀스 x가 되고 마지막 40개 문자는 타깃 시퀀스 y가 된다.  

Dataset 객체 `ds_tex_encoded`에 인코딩된 전체 텍스트를 원본 문자 순서대로 저장해 놓았다.  
먼저 `batch()` 메서드를 사용해서 41개의 문자로 구성된 텍스트 조각을 만든다. 즉, `batch|_size=41`로 지정한다. 마지막 배치 길이가 41보다 작으면 이 배치는 버린다. 따라서 만들어진 `ds_chunks` 데이터 셋은 항상 길이가 41인 시퀀스를 담고 있다. 이 41개의 문자 조각을 사용해서 시퀀스x(입력)와 시퀀스y(타깃)를 만든다. 두 시퀀스 모두 40개의 원소로 구성된다.

In [15]:
seq_length = 40
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

## inspection:
for seq in ds_chunks.take(1):
    input_seq = seq[:seq_length].numpy()
    target = seq[seq_length].numpy()
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])), 
          ' -> ', repr(''.join(char_array[target])))

[44 32 29  1 37 48 43 44 29 42 33 39 45 43  1 33 43 36 25 38 28  1  6  6
  6  0  0  0  0  0 40 67 64 53 70 52 54 53  1 51]  ->  74
'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'  ->  'y'


In [16]:
## x & y를 나누기 위한 함수를 정의합니다
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

## 확인:
for example in ds_sequences.take(2):
    print('입력 (x):', repr(''.join(char_array[example[0].numpy()])))
    print('타깃 (y):', repr(''.join(char_array[example[1].numpy()])))
    print()

입력 (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
타깃 (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

입력 (x): ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
타깃 (y): 'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'



2022-11-26 16:57:11.636353: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-26 16:57:11.636534: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


데이터셋 준비의 마지막 단계로 이 데이터셋을 미니 배치로 나눈다. 데이터셋을 배치로 나누기 위해 첫 번째 전처리 단계에서 문장 조각을 만들었다. 각 조각이 하나의 훈련 샘플에 대응하는 문장을 표현한다. 이제 훈련 샘플을 섞고 입력 미니 배치로 나눈다. 각 배치는 여러 개의 훈련 샘플을 가지고 있을 것이다. 

In [17]:
# 배치 크기
BATCH_SIZE = 64
BUFFER_SIZE = 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)# drop_remainder=True)

ds

<BatchDataset shapes: ((None, 40), (None, 40)), types: (tf.int32, tf.int32)>

#### 문자 수준의 RNN 모델 만들기  

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         20480     
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         1574912   
_________________________________________________________________
dense (Dense)                (None, None, 80)          41040     
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


이 모델의 LSTM 층의 출력 크기는 (None, None, 512)로 랭크 3이다.  
첫 번째 차원은 배치 차원이다. 두 번째 차원은 출력 시퀀스 길이이고 마지막 차원은 은닉 유닛의 개수에 해당한다.  
LSTM 층이 랭크 3의 출력을 만드는 이유는 이 LSTM 층을 만들 때 return_sequences=True로 지정했기 때문이다. 완전 연결 층(Dense)이 LSTM 층의 출력을 받아 출력 시퀀스와 각 원소마다 로짓을 계산한다. 결국 이 모델의 최종 출력도 랭크 3 텐서가 된다.  

또한, 마지막 완전 연결 층을 activation=None으로 설정했다. 새로운 텍스트를 생성하기 위해 모델 예측 값에서 샘플링할 수 있도록 로짓 출력이 필요하기 때문이다.

In [19]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))

model.fit(ds, epochs=20)

Epoch 1/20


2022-11-26 20:47:21.017758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-26 20:47:21.613752: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-26 20:47:22.494530: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


424/424 [==============================] - 20s 41ms/step - loss: 2.7852
Epoch 2/20
424/424 [==============================] - 18s 41ms/step - loss: 2.1869
Epoch 3/20
424/424 [==============================] - 18s 40ms/step - loss: 1.8629
Epoch 4/20
424/424 [==============================] - 18s 41ms/step - loss: 1.6510
Epoch 5/20
424/424 [==============================] - 18s 41ms/step - loss: 1.5136
Epoch 6/20
424/424 [==============================] - 18s 41ms/step - loss: 1.4328
Epoch 7/20
424/424 [==============================] - 18s 41ms/step - loss: 1.3706
Epoch 8/20
424/424 [==============================] - 18s 41ms/step - loss: 1.3262
Epoch 9/20
424/424 [==============================] - 18s 41ms/step - loss: 1.2968
Epoch 10/20
424/424 [==============================] - 18s 41ms/step - loss: 1.2716
Epoch 11/20
424/424 [==============================] - 18s 40ms/step - loss: 1.2467
Epoch 12/20
424/424 [==============================] - 18s 40ms/step - loss: 1.2249
Epoch 13/20


#### 평가 단계: 새로운 텍스트 생성
이전 절에서 훈련한 RNN 모델은 각 문자에 대해 80개 크기의 로짓을 반환한다. 소프트맥스 함수를 사용해서 이 로짓을 쉽게 확률로 바꿀 수 있다. 이 확률을 사용해서 어떤 문자가 다음에 올지 결정한다.  
시퀀스에서 다음 문자를 예측하기 위해 간단히 가장 큰 로짓 값을 가진 원소를 선택할 수 있다. 하지만 항상 가장 높은 확률을 가진 문자를 선택하는 대신 출력에서 샘플링하려고 한다. 이렇게 하지 않으면 모델이 항상 동일한 텍스트를 만든다. 텐서플로에서 제공하는 `tf.random.categorical()` 함수를 사용하여 범주형 분포에서 랜덤하게 샘플링할 수 있다. 어떻게 사용하는지 보기 위해 입력 로짓[1, 1, 1]일 때 세 개의 범주 [0, 1, 2]에서 랜덤하게 샘플링해보자

In [20]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 1.0]]
print('확률:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

확률: [0.33333334 0.33333334 0.33333334]
array([[0, 0, 1, 2, 0, 0, 0, 0, 1, 0]])


여기서 볼 수 있듯이 로짓이 같으므로 이 범주는 동일한 확률을 가진다.(즉, 범주의 선택 가능성이 동일하다.) 따라서 샘플 크기가 크면 각 범주가 등장할 횟수는 샘플 크기의 $\approx 1/3$에 이를 것으로 기대할 수 있다.  
로짓을 [1, 1, 3]으로 바꾸면 범주 2가 더 많이 등장할 것이다. 

In [21]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('확률:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

확률: [0.10650698 0.10650698 0.7869862 ]
array([[2, 0, 2, 2, 2, 0, 1, 2, 2, 0]])


`tr.random.categorical` 함수를 사용하면 모델이 출력한 로짓을 기반으로 문자를 생성할 수 있다. 짧은 시작 문자열 `starting_str`을 받아 새로운 `generated_str`을 생성하는 `sample()` 함수를 정의한다. `generated_st`r은 초기에 입력 값으로 설정된다.  
그다음 `generated_str`의 마지막에서 `max_input_length` 크기의 문자열을 선택하여 정수 시퀀스 `encoded_input`으로 인코딩한다. `encoded_input`을 RNN 모델에 전달하여 로짓을 계산한다. 이 RNN 모델의 마지막 순환 층에서 `return_sequences=True`로 설정했기 때문에 입력 시퀀스와 동일한 길이의 로짓 시퀀스가 출력된다. 따라서 RNN 모델의 출력에 있는 각 원소는 모델이 입력 시퀀스를 관찰한 후 다음 문자를 위한 로짓을 표현한다.  

여기서 출력 `logits`의 마지막 원소만 `tf.random.categorical()` 함수로 전달하여 새로운 샘플을 생성한다. 새로운 샘플을 문자로 변환하고 생성된 문자열 `generated_text` 끝에 추가하여 길이를 1만큼 늘린다. 그다음 이 과정을 반복한다. 지정한 문자 길이만큼 생성될 때까지 `generated_text`에서 마지막 `max_input_length`개의 문자를 선택하고 이를 사용하여 새로운 문자를 생성한다. 새로운 원소를 만들기 위해 생성된 시퀀스를 입력으로 사용하는 과정을 자기휘귀(auto-regression)라고 부른다. 

In [22]:
def sample(model, starting_str, 
           len_generated_text=500, 
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

The island may escape from collected. They were
establish, and answered, it would make use its effect
raised the cone.”

“There is he was not tried in a few minutes and she had
nothing one of the country, or at the engineer
had stated flent to the one, the colonists led the engineer had been aariinly tried, was
more carefully easier.”

“That was the “Dup
went ourselves, without home and the return deding was not forgottole. No, it would shook himself was signal house--would have been so at least base. The


결과에서 볼 수 있듯이 이 모델은 거의 정확한 단어를 생성한다. 몇몇 문장은 부분적으로 의미가 있다. 훈련 시 입력 시퀀스, 모델 구조, 샘플링 파라미터같은 훈련 파라미터를 더 튜닝해 볼 수 있다.  

생성된 샘플의 예측 가능성을 조절하기 위해(즉, 생성된 텍스트가 훈련 텍스트에서 학습한 패턴을 따르게 할지 랜덤하게 생성할지 조절하기 위해) RNN이 계산한 로짓을 `tf.random.categorical()` 샘플링 함수로 전달하기 전에 스케일을 조정할 수 있다. 스케일링 인자 $\alpha$를 물리학에 있는 온도의 역수로 해석할 수 있다. 온도가 높으면 무작위성이 커지고 온도가 낮으면 예측 가능한 행동을 만든다.  
$\alpha<1$로 로짓의 스케일을 조정하면 소프트맥스 함수가 계산할 확률은 다음 코드처럼 더 균일해진다. 

In [24]:
logits = np.array([[1.0, 1.0, 3.0]])

print('스케일 조정 전의 확률: ', tf.math.softmax(logits).numpy()[0])

print('0.5배 조정 후 확률:  ', tf.math.softmax(0.5*logits).numpy()[0])

print('0.1배 조정 후 확률:  ', tf.math.softmax(0.1*logits).numpy()[0])

스케일 조정 전의 확률:  [0.10650698 0.10650698 0.78698604]
0.5배 조정 후 확률:   [0.21194156 0.21194156 0.57611688]
0.1배 조정 후 확률:   [0.31042377 0.31042377 0.37915245]


여기서 볼 수 있듯이 $\alpha = 0.1$로 로짓의 스케일을 조정하면 거의 균등한 확률 [0.31, 0.31, 0.38]을 얻는다. 

In [25]:
# 알파=2.0 -> 예측 가능성이 높아짐
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=2.0))

The island was still a sign of men with the sea by a large quantity of which were still not only to the engineer, who was to be feared, and it was added in the mouth of the construction of the waters of the sailor’s explosion.

“The unfortunate mass were reached the engineer, “and there is no longer to him. “I am not a ready to be fired. The sailor would
not be able to do.”

“That is the convict could not be seen that the time was lost, the colonists were never
at the stream of the vessel. The colonists o


In [26]:
# 알파=0.5 -> 무작위성이 높아짐
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=0.5))

The island hairlaged
Ajpievat hemidus?” asked CMela..

Dub.? That’s’ howness,” chingl;
welcanoe.
Feeual, if it recoleous. Timplosed screecheagnzin,
if
Hlesson.”
Trub his fadder of,” of! ourb! Nr. Goy,!nBus.” ,ibser?’;,” sai, belt? qkige!
bignt up,” he?usall, bination, ansmesquil.
Framphy his
admirations! In,tR?
hopee  for keclo
Ia, it would
are, powi” or
when
Hebbslilvilynnspmindy an-inexplinewh
ap farbaruf clothes, untwelve,? Imrouse?” usiace bain)unwSutreak, 60lew opports of preserved
Gyinfy AnO5s took 


$\alpha =0.5$로 로짓의 스케일을 조정하면 더 랜덤한 텍스트가 생성된다. 올바른 텍스트와 신선한 텍스트 생성 사이에서 절충점을 찾아야 한다. 

# 트랜스포머 모델을 사용한 언어 이해

이 장에서 RNN 기반의 신경망으로 두 가지 시퀀스 모델링 문제를 풀었다. 하지만 최근에 새롭게 등장한 한 모델 구조가 여러 NLP 작업에서 RNN 기반의 seq2seq 모델의 성능을 능가하는 것으로 나타났다.  

이 구조가 **트랜스포머**(Transformer)이다. 2017년 NeurIPS 논문에서 소개되었으며 입력과 출력 시퀀스 사이에 있는 전역 의존성(global dependency)을 모델링할 수 있다. 트랜스포머 구조는 **어텐션**(attention)이라는 개념을 기반으로 한다. 좀 더 구체적으로는 **셀프 어텐션 메커니즘**(self-attention mechanism)을 기반으로 한다.

## 셀프 어텐션 메커니즘 이해

#### 셀프 어텐션 기본 구조
셀프 어텐션의 기본 아이디어를 소개하기 위해 길이가 T인 입력 시퀀스 $x^{(0)}, x^{(1)}, \cdots ,x^{(T)}$와 출력 시퀀스 $o^{(0)}, o^{(1)}, \cdots, o^{(T)}$가 있다고 가정하자. 시퀀스의 각 원소 $x^{(t)}$와 $o^{(t)}$는 크기가 d인 벡터이다. 그다음 seq2seq 작업에서 셀프 어텐션은 입력 원소에 대한 출력 시퀀스에 있는 각 원소의 의존성을 모델링하는 것이 목적이다. 이를 위해 어텐션 메커니즘은 세 단계로 구성된다.  
첫째 현재 원소와 시퀀스에 있는 다른 모든 원소 사이의 유사도를 기반으로 중요도 가중치를 계산한다. 둘째 익숙한 소프트맥스 함수르 사용하여 이 가중치를 정규화한다. 셋째 이 가중치를 해당하는 시퀀스 원소와 결합하여 어텐션 값을 계산한다.  

정리하면 셀프 어텐션 연산의 주요 세 단계는 다음과 같다.  
- 주어진 입력 원소 $x^{(i)}$와 [0, T] 범위에 있는 j번째 원소에 대해 점곱 ${x^{(i)}}^{T}x^{(j)}$를 계산한다.  
- 소프트맥스 함수로 이 점곱을 정규화하여 가중치 $W_{ij}$를 얻는다.  
- 전체 입력 시퀀스에 대한 가중치 합으로 출력 $o^{(i)}$를 계산한다.  
$$o^{(i)}=\sum^{T}_{j=0}W_{ij}x^{(j)}$$

In [27]:
Image(url='https://git.io/JLdVo', width=700)

#### 쿼리, 키, 값 가중치를 가진 셀프 어텐션 메커니즘
이 절에서는 트랜스포머 모델에서 사용하는 고급 셀프 어텐션 메커니즘을 정리해 본다. 이전 절에서는 출력을 계산할 때 학습되는 파라미터를 전혀 사용하지 않았다. 따라서 언어 모델을 훈련할 때 분류 오차를 최소화하는 것 같이 목적 함수를 최적화하려면 입력 원소 $x^{(i)}$가 되는 단어 임베딩(즉, 입력 벡터)을 바꾸어야 한다. 다르게 말해 앞서 소개한 기본적인 셀프 어텐션 메커니즘을 사용하면 트랜스포머 모델이 주어진 시퀀스에서 모델을 최적화하는 동안 어텐션 값을 바꾸거나 업데이트하는 데 제한적이다.  
셀프 어텐션 메커니즘을 모델 최적화에 대해 유연하고 적응할 수 있게 만들기 위해 추가적인 가중치 행렬을 사용한다. 이 가중치는 모델을 훈련하는 동안 학습되는 모델 파라미터이다.  
이 세 가중치 행렬을 $U_{q}, U_{k}, U_{v}$로 표시한다. 이 가중치는 입력을 쿼리(query), 키(key), 값(value) 시퀀스로 만들기 위해 사용된다.  

- 쿼리 시퀀스: $q^{(i)}=U_{q}x^{(i)} \qquad i \in [0, T]일\;때$  
- 키 시퀀스: $k^{(i)}=U_{k}x^{(i)} \qquad i \in [0, T]일\;때$  
- 값 시퀀스: $v^{(i)}=U_{v}x^{(i)} \qquad i \in [0, T]일\;때$   

In [28]:
Image(url='https://git.io/JLdV6', width=700)

위의 그림은 트랜스포머 블록을 나타낸 것이다. 